## Initial Setup

In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1663132592206_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Basics").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Add all import commands to this cell. 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, types
from pyspark.sql.functions import col, regexp_replace, lit, split
#import sys
#import string
from pyspark.sql.types import StructType, FloatType, ArrayType, StringType



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the data

In [5]:
raw_recipes_df = spark.read.csv("s3a://raw-recipes-clean-upgrad/RAW_recipes_cleaned.csv", 
                                 header=True, inferSchema=True
                               )

# Please forward the exact name of data frames and columns as suggested in the code. 
# It will ensure that the assert commands function correctly.  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert raw_recipes_df.count() == 231637, "There is a mistake in reading the data."
assert len(raw_recipes_df.columns) == 12, "There is a mistake in reading the data."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Extract ```nutrition``` values 

In [7]:
raw_recipes_df.select('nutrition').show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------+
|nutrition                                 |
+------------------------------------------+
|[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]     |
|[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0] |
|[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]|
|[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0] |
|[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0] |
+------------------------------------------+
only showing top 5 rows

In [8]:
# List of nutrition columns

nutrition_column_names = ['calories',
                          'total_fat_PDV',
                          'sugar_PDV',
                          'sodium_PDV',
                          'protein_PDV',
                          'saturated_fat_PDV',
                          'carbohydrates_PDV']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# STEP 1 - string operations to convert nutrition to an array 


raw_recipes_df = (raw_recipes_df
                  .withColumn("nutrition",  regexp_replace("nutrition", "\[", ""))
                  .withColumn("nutrition",  regexp_replace("nutrition", "\]", ""))
                  .withColumn('nutrition',  split(col("nutrition"),",")  ) # add command(s) to modify the string
                  )

#split the text 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
raw_recipes_df.select('nutrition').show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------+
|nutrition                                       |
+------------------------------------------------+
|[51.5,  0.0,  13.0,  0.0,  2.0,  0.0,  4.0]     |
|[173.4,  18.0,  0.0,  17.0,  22.0,  35.0,  1.0] |
|[269.8,  22.0,  32.0,  48.0,  39.0,  27.0,  5.0]|
|[368.1,  17.0,  10.0,  2.0,  14.0,  8.0,  20.0] |
|[352.9,  1.0,  337.0,  23.0,  3.0,  0.0,  28.0] |
+------------------------------------------------+
only showing top 5 rows

In [11]:
# STEP 2 - create new columns with individual nutrition values 

# Also eusure that new columns are float type

raw_recipes_df = (raw_recipes_df
                  .withColumn('calories', raw_recipes_df.nutrition[0].cast('float'))
                  .withColumn('total_fat_PDV', raw_recipes_df.nutrition[1].cast('float'))
                  .withColumn('sugar_PDV', raw_recipes_df.nutrition[2].cast('float'))
                  .withColumn('sodium_PDV', raw_recipes_df.nutrition[3].cast('float'))
                  .withColumn('protein_PDV', raw_recipes_df.nutrition[4].cast('float'))
                  .withColumn('saturated_fat_PDV', raw_recipes_df.nutrition[5].cast('float'))
                  .withColumn('carbohydrates_PDV', raw_recipes_df.nutrition[6].cast('float'))
                 )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert raw_recipes_df.schema["carbohydrates_PDV"].dataType == FloatType(), "Recheck your typecasting"
assert raw_recipes_df.collect()[123432][14] == 62.0, "The columns have not been split correctly."
assert raw_recipes_df.collect()[10000][12] == 60.400001525878906, "The columns have not been split correctly."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Make nutrition-per-100 calorie columns

By converting the nutrition values from absolute to relative terms, we ensure that portion size is not a factor in the analysis. 


Naming convention: Original column name ```total fat (PDV)```, column name after column ```total_fat_per_100_cal```

In [13]:
for nutrition_col in nutrition_column_names:
    if nutrition_col != 'calories':
        nutrition_per_100_cal_col = (nutrition_col
                                 .replace('_PDV','')
                                 +'_per_100_cal')
        # Write code to create nutrition columns per 100 calories 
nutrition_column_names
        
        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['calories', 'total_fat_PDV', 'sugar_PDV', 'sodium_PDV', 'protein_PDV', 'saturated_fat_PDV', 'carbohydrates_PDV']

In [14]:
from pyspark.sql.functions import udf
#https://stackoverflow.com/questions/48282321/valueerror-cannot-convert-column-into-bool

@udf("double")    
def nutritonPercent(x,y):
    if y <= 0.0:
        return 0.0
    return ((x / y) * 100)

raw_recipes_df = (raw_recipes_df
                  .withColumn('total_fat_per_100_cal', nutritonPercent(raw_recipes_df.total_fat_PDV, raw_recipes_df.calories))
                  .withColumn('sugar_per_100_cal', nutritonPercent(raw_recipes_df.sugar_PDV, raw_recipes_df.calories))
                  .withColumn('sodium_per_100_cal',  nutritonPercent(raw_recipes_df.sodium_PDV, raw_recipes_df.calories))
                  .withColumn('protein_per_100_cal',  nutritonPercent(raw_recipes_df.protein_PDV, raw_recipes_df.calories))
                  .withColumn('saturated_fat_per_100_cal',  nutritonPercent(raw_recipes_df.saturated_fat_PDV, raw_recipes_df.calories))
                  .withColumn('carbohydrates_per_100_cal',  nutritonPercent(raw_recipes_df.carbohydrates_PDV, raw_recipes_df.calories))
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# total fat check for id 28881
assert raw_recipes_df.filter("id == 28881").select('total_fat_per_100_cal').first()[0] == 0, "total_fat_per_100_cal for recipe 28881 should be 0"

# total fat check for id 112140
assert round(raw_recipes_df.filter("id == 112140").select('total_fat_per_100_cal').first()[0]) == 8, "total_fat_per_100_cal for recipe 112140 should be 8"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
raw_recipes_df2 = raw_recipes_df
raw_recipes_df2.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+-----------------+------------------+-------------------+-------------------------+-------------------------+
|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal|sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+-----------

## Convert `Tags` to `array` type

In [41]:
raw_recipes_df2.select('tags').show(1, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tags                                                                                                                                                                                                                                                                      |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-eve

In [28]:
raw_recipes_df2.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('name', 'string'), ('id', 'int'), ('minutes', 'int'), ('contributor_id', 'int'), ('submitted', 'timestamp'), ('tags', 'string'), ('nutrition', 'array<string>'), ('n_steps', 'int'), ('steps', 'string'), ('description', 'string'), ('ingredients', 'string'), ('n_ingredients', 'int'), ('calories', 'float'), ('total_fat_PDV', 'float'), ('sugar_PDV', 'float'), ('sodium_PDV', 'float'), ('protein_PDV', 'float'), ('saturated_fat_PDV', 'float'), ('carbohydrates_PDV', 'float'), ('total_fat_per_100_cal', 'double'), ('sugar_per_100_cal', 'double'), ('sodium_per_100_cal', 'double'), ('protein_per_100_cal', 'double'), ('saturated_fat_per_100_cal', 'double'), ('carbohydrates_per_100_cal', 'double')]

In [47]:
raw_recipes_df2 = (raw_recipes_df
                  #.withColumn("tags",  regexp_replace("tags", "\[", ""))
                  #.withColumn("tags",  regexp_replace("tags", "\]", ""))
                  #.withColumn("tags",  regexp_replace("tags", "\"", ""))
                  #.withColumn("tags",  regexp_replace("tags", "\]\"", ""))
                  #.withColumn("tags",  regexp_replace("tags", "|", ""))
                  .withColumn("tags",  regexp_replace("tags", "\"", ""))
                  .withColumn('tags', split(col("tags"),",")) # code to manipulate strings 
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
raw_recipes_df2.select('tags').show(1, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tags                                                                                                                                                                                                                                                                                           |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[['60-minutes-or-less',  'time-to-make',  'course',  'main-ingredient',  'cuisine',  'preparation',  'occasion',  'north-american

In [45]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert raw_recipes_df.schema["tags"].dataType == ArrayType(StringType(), True), "You have not split the string into an array."
assert raw_recipes_df.collect()[2][5] == ['time-to-make','course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less'], "Recheck your string cleaning and splitting operations."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
You have not split the string into an array.
Traceback (most recent call last):
AssertionError: You have not split the string into an array.



In [49]:
raw_recipes_df.collect()[2][5]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']"

In [22]:
raw_recipes_df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('name', 'string'), ('id', 'int'), ('minutes', 'int'), ('contributor_id', 'int'), ('submitted', 'timestamp'), ('tags', 'array<string>'), ('nutrition', 'array<string>'), ('n_steps', 'int'), ('steps', 'string'), ('description', 'string'), ('ingredients', 'string'), ('n_ingredients', 'int'), ('calories', 'float'), ('total_fat_PDV', 'float'), ('sugar_PDV', 'float'), ('sodium_PDV', 'float'), ('protein_PDV', 'float'), ('saturated_fat_PDV', 'float'), ('carbohydrates_PDV', 'float'), ('total_fat_per_100_cal', 'double'), ('sugar_per_100_cal', 'double'), ('sodium_per_100_cal', 'double'), ('protein_per_100_cal', 'double'), ('saturated_fat_per_100_cal', 'double'), ('carbohydrates_per_100_cal', 'double')]

## Join Recipe Data to Review Data

In [19]:
# Reading the second data set. 
# keep this cell unedited

raw_ratings_df = (spark.read.csv("s3a://raw-interactions-upgrad/RAW_interactions_cleaned.csv", 
                                # add arguments
                                )
                  .withColumn("review_date",  F.col("date"))
                  .drop(F.col("date"))
                  )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
raw_ratings_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: string (nullable = true)

In [21]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert raw_ratings_df.count() == 1132367, "There is a mistake in reading the data."
assert len(raw_ratings_df.columns) == 5, "There is a mistake in reading the data."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
raw_ratings_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-----------+
|user_id|recipe_id|rating|              review|review_date|
+-------+---------+------+--------------------+-----------+
|  38094|    40893|     4|Great with a sala...| 2003-02-17|
|1293707|    40893|     5|So simple  so del...| 2011-12-21|
|   8937|    44394|     4|This worked very ...| 2002-12-01|
| 126440|    85009|     5|I made the Mexica...| 2010-02-27|
|  57222|    85009|     5|Made the cheddar ...| 2011-10-01|
+-------+---------+------+--------------------+-----------+
only showing top 5 rows

In [23]:
interaction_level_df = raw_ratings_df.join(
                                            # code to join raw_recipes to raw_reviews
                                           )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

list1 = raw_ratings_df.select('recipe_id').collect()
list2 = raw_recipes_df.select('id').collect()
exclusive_set = set(list1)-set(list2)

assert len(exclusive_set) == 0, "There is a mistake in reading one of the two data files."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
interaction_level_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-----------+--------------------+---+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+-------------------+------------------+-------------------+-------------------------+-------------------------+
|user_id|recipe_id|rating|              review|review_date|                name| id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal|  sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+-------+---------+------+--------------------+-----------

## Recipe Submission Date Based Features

**Steps:**
1. Convert ```submitted``` column to `DateType()`
2. Use review date to derive columns `days_since_submission_on_review_date`,  `months_since_submission_on_review_date`,  `years_since_submission_on_review_date`

In [26]:
interaction_level_df = (interaction_level_df
                        .withColumn('submitted', # type casting code 
                                   ))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
interaction_level_df = (interaction_level_df
                        .withColumn('days_since_submission_on_review_date',
                                    # code to calculate days between review and submission
                                   )
                        .withColumn('months_since_submission_on_review_date',
                                    # code to calculate months between review and submission           
                                   )
                        .withColumn('years_since_submission_on_review_date',
                                    # code to calculate years between review and submission          
                                   ))

In [27]:
interaction_level_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-----------+--------------------+---+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+-------------------+------------------+-------------------+-------------------------+-------------------------+------------------------------------+--------------------------------------+-------------------------------------+
|user_id|recipe_id|rating|              review|review_date|                name| id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal|  sugar_per_100_cal|sodium_per_100_cal|protein_per_100_c

## Save the data we have created so far in a parquet file. 

In [28]:
interaction_level_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: date (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturated_fat_PDV: float (nullable = true)
 |-- carb

In [33]:
## Write the raw_recipes_df
## Write data to the S3 bucket which will be your working directory 

interaction_level_df.write.mode('overwrite').parquet( 
                                                    # add code here 
                                                     )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…